<H1> Capstone Project Notebook <H1>

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<B>Part 1<B>

In [3]:
#!conda install -c conda-forge lxml --yes #Install LXML package to use with Pandas HTML parser.
#!conda install -c conda-forge folium=0.5.0 --yes #Install Folium to draw maps

In [4]:
from bs4 import BeautifulSoup 
pd.options.mode.chained_assignment = None #disable chained warnings for Pandas
pd.set_option('display.max_columns', None) #remove length restrictions
pd.set_option('display.max_rows', None) #remove width restrictions

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans #import k-means from clustering stage

import folium # map rendering library

In [5]:
#Data sources
WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
GEODATA_CSV = 'https://cocl.us/Geospatial_data'

#Toronto coordinates
TORONTO_LAT = 43.70011
TORONTO_LONG = -79.4163

#Foursquare parameters
LIMIT = 100
RADIUS = 500

#Foursquare API URL templates
FS_EXPLORE_URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'


In [6]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0]) 

In [8]:
neighborhood_data = neighborhood[neighborhood['Borough'] != 'Not assigned'].reset_index(drop=True)
toronto_neighborhood = neighborhood_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_neighborhood.shape

(103, 3)